In [1]:
from neo4j import GraphDatabase
import pandas as pd

# 接続、初期化

In [2]:
uri = 'bolt://neo4j:7687'
user = 'neo4j'
password = 'neo4j'
driver = GraphDatabase.driver(uri, auth=(user, password))

In [3]:
def clear_db(tx):
    '''
    ノードと関係性を全て削除する
    '''
    tx.run('MATCH (n) DETACH DELETE n')

def get_constrs(tx):
    '''
    制約名リストを取得する
    '''
    constrs = tx.run('CALL db.constraints')
    constrs_name = []
    for constr in constrs:
        constrs_name.append(constr.value('name'))
    return constrs_name
    
def drop_constrs(tx, constrs):
    '''
    制約を削除する
    '''
    if not isinstance(constrs, list):
        constrs = [constrs]
    for constr in constrs:
        tx.run('DROP CONSTRAINT ' + constr)

In [4]:
# ノード削除
with driver.session() as session:
    session.write_transaction(clear_db)
# 制約削除
with driver.session() as session:
    constrs = session.read_transaction(get_constrs)
    session.write_transaction(drop_constrs, constrs)

# 作業順序への適用

+ パターン毎の作業順序をグラフDB化
+ 工程マスタ、作業マスタ、作業順序を兼ねられる？？
    + 「ある工程の全作業」「ある作業が属する工程」など取得する関数を作ればOK
    + 外部キー制約等は付けられないからダメ？？

## neo4jへのデータ登録

In [5]:
WORK = '作業'
KOUTEI = '工程'
PATTERN = 'パターン'
COLNAME_ORDER_ID = '作業順序ID'
COLNAME_PRETASK = '前作業'
COLNAME_POSTTASK = '後作業'
COLNAME_PRETASK_ORIGIN = '前作業原点'
COLNAME_POSTTASK_ORIGIN = '後作業原点'
COLNAME_OFFSET_TIME = '時間差'
COLNAME_CONSTR_TYPE = '制約種類'

In [6]:
path_sagyou_master = './data/sagyou_master_demo.xlsx'
path_sagyou_order = './data/sagyou_order_demo.xlsx'

df_sagyou_master = pd.read_excel(path_sagyou_master)
df_sagyou_order = pd.read_excel(path_sagyou_order)

In [7]:
def add_unique_sagyou(tx):
    '''
    作業ノードの「作業」属性にUNIQUE制約付与
    '''
    tx.run('CREATE CONSTRAINT ON (w:Work) ASSERT w.work IS UNIQUE')

def add_unique_sagyou_order(tx):
    '''
    作業順序リレーションの「パターン」「作業順序ID」にUNIQUE制約付与
    （有料版でないと効かない？）
    '''
    tx.run('CREATE CONSTRAINT ON (o:ORDER) ASSERT (o.pattern, o.order_id) IS UNIQUE')

def add_sagyou(tx, df):
    '''
    作業ノードの登録
    各行につき1ノードとし、列「作業」を属性として追加
    '''
    for _, ds in df.iterrows():
        tx.run(f'CREATE (:Work:{ds[KOUTEI]} {{work: $work}})', work=ds[WORK])

def add_sagyou_order(tx, df):
    '''
    作業順序リレーションを登録
    各行につき1リレーションとし、各列（前作業、後作業以外）を属性として追加
    '''
    for _, ds in df.iterrows():
        cypher = '''
            MATCH
                (a:Work{work:$pretask}),(b:Work{work:$posttask})
            CREATE
                (a)-[:ORDER{
                    pattern: $pattern,
                    order_id: $order_id,
                    pretask_origin: $pretask_origin,
                    posttask_origin: $posttask_origin,
                    offset_time: $offset_time,
                    constr_type: $constr_type
                }]->(b);
        '''
        tx.run(
            cypher,
            pretask=ds[COLNAME_PRETASK],
            posttask=ds[COLNAME_POSTTASK],
            pattern=ds[PATTERN],
            order_id=ds[COLNAME_ORDER_ID],
            pretask_origin=ds[COLNAME_PRETASK_ORIGIN],
            posttask_origin=ds[COLNAME_POSTTASK_ORIGIN],
            offset_time=ds[COLNAME_OFFSET_TIME],
            constr_type=ds[COLNAME_CONSTR_TYPE]
        )

In [8]:
# 制約追加
with driver.session() as session:
    session.write_transaction(add_unique_sagyou)
    session.write_transaction(add_unique_sagyou_order)

In [9]:
# ノード、リレーション追加
with driver.session() as session:
    session.write_transaction(add_sagyou, df_sagyou_master)
    session.write_transaction(add_sagyou_order, df_sagyou_order)

## データの取得

In [10]:
# 作業の取得
def get_work_node_in_koutei(tx, koutei):
    '''
    ある工程に存在する:Workノードを取得する
    
    koutei : str or list
    '''
    node_list = []
    # kouteiがリストなら、各要素に対してこの関数を適用
    if isinstance(koutei, list):
        for k in koutei:
            node_list += get_work_node_in_koutei(tx, k)
    else:
        # データを取得
        records = tx.run(f'MATCH (w:Work:{koutei}) RETURN w;',
                         koutei=koutei)
        for record in records:
            node_list.append(record)
    return node_list


def get_work_in_koutei(tx, koutei):
    '''
    ある工程に存在する作業名を取得する
    
    koutei : str or list
    '''
    node_list = get_work_node_in_koutei(tx, koutei)
    work_list = [node['w']['work'] for node in node_list]
    return work_list

In [11]:
# 作業順序の取得
def get_work_order_node(tx, koutei=None, pattern=None):
    '''
    ある工程、パターンに存在する:Workノードと:ORDERリレーションを取得する
    
    koutei,pattern : str,list or optional
        str,listで指定する。Noneの場合は指定しない
    '''
    node_list = []
    # patternがリストなら、各要素に対してこの関数を適用
    if isinstance(pattern, list):
        for p in pattern:
            node_list += get_work_order_node(tx, koutei=koutei, pattern=p)
    else:
        # 属性値の選択
        if koutei is None:
            where_koutei = ''
        else:
            koutei = koutei if isinstance(koutei, list) else [koutei]
            where_koutei = 'WHERE '
            for k in koutei:
                where_koutei += f'(w1:{k} AND w2:{k}) OR '
            where_koutei = where_koutei[:-4]
            
        select_pattern = '' if pattern is None else '{pattern: $pattern}'
        kwargs_select = {} if pattern is None else {'pattern': pattern}

        # データを取得
        cypher = f'''
            MATCH
                (w1:Work)-[o:ORDER{select_pattern}]->(w2:Work)
            {where_koutei}
            RETURN
                w1,o,w2
        '''
        records = tx.run(cypher, **kwargs_select)
        for record in records:
            node_list.append(record)
    
    return node_list


def get_work_order_df(tx, koutei=None, pattern=None):
    '''
    ある工程、パターンに存在する作業順序情報を取得する
    
    koutei,pattern : str,list or optional
        str,listで指定する。Noneの場合は指定しない
    '''
    node_list = get_work_order_node(tx, koutei=koutei, pattern=pattern)
    dict_order = [
        {
            PATTERN: node['o']['pattern'],
            COLNAME_ORDER_ID: node['o']['order_id'],
            COLNAME_PRETASK: node['o'].nodes[0]['work'],
            COLNAME_POSTTASK: node['o'].nodes[1]['work'],
            COLNAME_PRETASK_ORIGIN: node['o']['pretask_origin'],
            COLNAME_POSTTASK_ORIGIN: node['o']['posttask_origin'],
            COLNAME_OFFSET_TIME: node['o']['offset_time'],
            COLNAME_CONSTR_TYPE: node['o']['constr_type'],
        }
        for node in node_list
    ]
    
    df_order = pd.DataFrame(dict_order)
    return df_order

In [12]:
# 作業名取得
with driver.session() as session:
    work_list = session.read_transaction(get_work_in_koutei, ['工程A', '工程B'])

In [13]:
work_list

['作業3', '作業4', '作業5', '作業1', '作業2', '作業6', '作業7', '作業8']

In [14]:
# 作業順序取得
with driver.session() as session:
    df_order_get = session.read_transaction(
        get_work_order_df, koutei=['工程A', '工程B'], pattern='A-1')

In [15]:
df_order_get

,パターン,作業順序ID,前作業,後作業,前作業原点,後作業原点,時間差,制約種類
0,A-1,2,作業1,作業3,終了時刻,開始時刻,0,ぴったり
1,A-1,4,作業3,作業5,終了時刻,開始時刻,0,いつでも
2,A-1,3,作業2,作業5,終了時刻,開始時刻,10,いつでも
3,A-1,6,作業6,作業7,終了時刻,開始時刻,30,ぴったり
4,A-1,1,作業1,作業2,終了時刻,開始時刻,0,いつでも


Browser

+ あるパターンの順序
    + MATCH (w1:Work)-[r:ORDER{pattern:"A-1"}]->(w2:Work) RETURN w1,r,w2;
+ あるパターン、ある工程の順序
    + MATCH
        (w1:Work:工程A)-[o:ORDER{pattern:"A-1"}]->(w2:Work:工程A)
    RETURN
        w1,o,w2